In [1]:
import torch
from torch import nn, optim

from models.mtcnn import Mtcnn
from dataloader import DataLoader

In [2]:
EPOCHS = 30
ETA = 1e-3
BATCH_SIZE = 128
MODEL_PATH = "saved/mtcnn-20200118.pth"

In [3]:
model = Mtcnn()

# load
model.load(MODEL_PATH)

train_loader = DataLoader(BATCH_SIZE, True)
valid_loader = DataLoader(BATCH_SIZE, False)

MTCNN was loaded.


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)

In [5]:
top_valid_acc = 0.0

for e in range(EPOCHS):
    train_loss = 0.0
    train_p_face_acc = 0.0
    train_r_face_acc = 0.0
    train_o_face_acc = 0.0
    
    valid_loss = 0.0
    valid_p_face_acc = 0.0
    valid_r_face_acc = 0.0
    valid_o_face_acc = 0.0
    
    for img, face, bbox, lm in train_loader.next_batch():
        loss, p_face_acc, r_face_acc, o_face_acc = model([img, face, bbox, lm], training=True)
        
        train_loss += loss.item()
        train_p_face_acc += p_face_acc.item()
        train_r_face_acc += r_face_acc.item()
        train_o_face_acc += o_face_acc.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        for img, face, bbox, lm in valid_loader.next_batch():
            loss, p_face_acc, r_face_acc, o_face_acc = model([img, face, bbox, lm], training=True)
            
            valid_loss += loss.item()
            valid_p_face_acc += p_face_acc.item()
            valid_r_face_acc += r_face_acc.item()
            valid_o_face_acc += o_face_acc.item()
        
    train_loss /= len(train_loader)
    train_p_face_acc /= len(train_loader)
    train_r_face_acc /= len(train_loader)
    train_o_face_acc /= len(train_loader)
    
    valid_loss /= len(valid_loader)
    valid_p_face_acc /= len(valid_loader)
    valid_r_face_acc /= len(valid_loader)
    valid_o_face_acc /= len(valid_loader)
    
    print(f"Epochs {e+1}/{EPOCHS}")
    
    print(f"Train loss: {train_loss:.8f}")
    print(f"Train PNet face accuracy: {train_p_face_acc:.4f}")
    print(f"Train RNet face accuracy: {train_r_face_acc:.4f}")
    print(f"Train ONet face accuracy: {train_o_face_acc:.4f}")
    
    print(f"Valid loss: {valid_loss:.8f}")
    print(f"Valid PNet face accuracy: {valid_p_face_acc:.4f}")
    print(f"Valid RNet face accuracy: {valid_r_face_acc:.4f}")
    print(f"Valid ONet face accuracy: {valid_o_face_acc:.4f}")
    
    mean_valid_acc = (valid_p_face_acc + valid_r_face_acc + valid_o_face_acc) / 3
    
    if top_valid_acc < mean_valid_acc:
        top_valid_acc = mean_valid_acc
        model.save(MODEL_PATH)

Epochs 1/30
Train loss: 0.39938139
Train PNet face accuracy: 0.9660
Train RNet face accuracy: 0.9754
Train ONet face accuracy: 0.9832
Valid loss: 0.37469024
Valid PNet face accuracy: 0.9660
Valid RNet face accuracy: 0.9799
Valid ONet face accuracy: 0.9844
MTCNN was saved.
Epochs 2/30
Train loss: 0.37729592
Train PNet face accuracy: 0.9692
Train RNet face accuracy: 0.9791
Train ONet face accuracy: 0.9827
Valid loss: 0.35915510
Valid PNet face accuracy: 0.9661
Valid RNet face accuracy: 0.9796
Valid ONet face accuracy: 0.9832
Epochs 3/30
Train loss: 0.38605166
Train PNet face accuracy: 0.9653
Train RNet face accuracy: 0.9768
Train ONet face accuracy: 0.9825
Valid loss: 0.34879026
Valid PNet face accuracy: 0.9703
Valid RNet face accuracy: 0.9811
Valid ONet face accuracy: 0.9871
MTCNN was saved.
Epochs 4/30
Train loss: 0.38031253
Train PNet face accuracy: 0.9658
Train RNet face accuracy: 0.9770
Train ONet face accuracy: 0.9840
Valid loss: 0.37056427
Valid PNet face accuracy: 0.9691
Valid RN

KeyboardInterrupt: 